# Policy Gradient Methods 

## Yue Dong & Ali Emami
In this assignment, Ali and I experimented the policy gradient methods on Mountain car problem, as suggested in chapter 13 of the textbook. We first implemented the REINFORCE which is a policy gradient algorithm based on the complete return as in Monte Carlo algorithm. We compared the result of REINFORCE with or without the baseline. We then implemented the actor-critic methods with 1-step return or with eligibility trace. **Note that we cut off the learning in each episode if time_step is over 200** in all algorithms for faster learning.

## 1. Preliminaries

In policy gradient learning, instead of using $Q(s,a)$ to choose the best action in a certain state, we select an action based on the *learned parameterized policy* $\pi(a | s,\mathbb{\theta})$. In more details, the action a is chosen with the probability $\pi(a | s,\mathbb{\theta})$ given the state $s$ at time $t$ with the weight vector $\theta$.

Although selecting actions is not based on consulting a value function in policy gradient algorithm, sometimes, we still need a parameterized value function. For example, in REINFORCE, we need $V$ as the baseline and in actor-critic with 1-step or n-steps or $\lambda$-returns, we need $V$ to form the return.  We therefore could use function approximation to parameterize the value function with respect to the weights $w$. Thus, $\hat{V}=V(s,w)$.

### (a) policy approximation 
Since the action space is discrete in both mountain car and easy21, we from the parameterization of the policy with the preferences $h(s,a, \theta)$ which is similar to the case in multi-arm bandit. Then **the policy** is defined as an exponential softmax distribution (h bigger for $a$, $a$ is more likely to be chosen):
$$\pi(a|s,\theta)=\frac{exp(h(s,a,\theta))}{\sum_b exp(h(s,b,\theta))}, b\in A_s$$

Here we choose a linear function approximation in features to represent the **preferences**: 
$$h(s,a,\theta) = \theta^{T} \phi(s,a)$$ where **the features** $\phi(s,a) \in \mathbb{R}^n$ is constructed as follows:

####  In mountain car,  we define  $\phi(s,a)$  as a binary feature vector obtained from tile coding with  (4∗9∗9)*3  features. 
we divide the 2D space into an 8x8 grid and then we offset it 3 times with 1/4 of a tile size to form 4 tilings.  We add one extra row and one extra column so that every point is covered by each tiling. A simple tile coding with offset is demonstraded as the following graph from Sutton's textbook. 

### Advantages of using policy parameterization over action-value parameterization:
-  the approximate policy can approach determinism rather than $\epsilon$ greedy algorithm.
-  useful when the policy is a simpler function to approximate than action-value function.

### The goal of policy learning here is to modify $\theta$ such that the performance measure $\eta(\theta)$ is maximized.
$$\theta_{t+1}=\theta_{t}+\alpha \nabla \eta  (\theta_t)$$

If we use $\eta(\theta)=v_{\pi_\theta}(s_0)$, then the gradient of $\eta(\theta)$ could be represented by 
<img src="policy_gradient_thm.png" style="max-width:50%; width: 50%; max-width: none">
Which is called **the policy gradient theorem**.

## 2. REINFORCE: Monte Carlo Policy Gradient
Using the policy gradient theorem, we could deduce that 
$$\nabla \eta(\theta) = E_{\pi}\big[\gamma^tG_t\frac{\nabla_{\theta}\pi(A_t|S_t,\theta)}{\pi(A_t|S_t,\theta)}\big]$$
We sample $G_t$ on each time step and define the update as: 
<img src="reinforce.png" style="max-width:50%; width: 50%; max-width: none">
Since $G_t$ is the complete return from time $t$, REINFORCE is a Monte Carlo method.

### (a) REINFORCE without baseline
<img src="reinforce_psedo.png" style="max-width:80%; width: 80%; max-width: none">
where $$\nabla_{\theta}log\pi(A_t|S_t,\theta) =  \frac{\nabla_{\theta}\pi(A_t|S_t,\theta)}{\pi(A_t|S_t,\theta)}$$ 
and with linear action preferences, 
<img src="reinforce_update.png" style="max-width:50%; width: 50%; max-width: none">

The code of REINFORCE without baseline could be found in **reinforce_mountain_car_log.py**. The following is the plot of 1000 episodes of 3 runs average. We use the running mean of interval 100 to make the plot looks smoother. 

<img src="reinforce_plot_no_baseline.png" style="max-width:60%; width: 60%; max-width: none">

### (b) REINFORCE with baseline
<img src="reinforce_baseline_psedo.png" style="max-width:60%; width: 60%; max-width: none">

The code of REINFORCE with baseline could be found in **reinforce_mountain_car_log_baseline.py**. The following is the plot of 1000 episodes of 3 runs average. We use the running mean of interval 100 to make the plot looks smoother. 

<img src="reinforce_with_baseline.png" style="max-width:60%; width: 60%; max-width: none">

## 3. Actor-Critic Methods

### (a) actor-critic with one step return
Instead of using monte carlo method, which learns very slow, We could use actor-critic with one step return. The error is the TD(0) error and the algorithm is updated online. 
<img src="ac_one_step_psedo.png" style="max-width:60%; width: 60%; max-width: none">

The code of one step actor-critic algorithm could be found in **ac_one_step.py**. The following is the plot of 200 episodes of 5 runs average. We use the running mean of interval 20 to make the plot looks smoother. 
<img src="ac_one_step.png" style="max-width:60%; width: 60%; max-width: none">

### (b) actor-critic with eligibility trace
To improve the learning performance, we tried actor-critic method with eligibility traces. Apparently this is the best algorithm so far in learning the mountain car. 
<img src="ac_e_trace_psedo.png" style="max-width:50%; width: 50%; max-width: none">
The code of actor-critic algorithm with eligibility trace could be found in **ac_e_trace.py**. The following is the plot of 200 episodes of 5 runs average. We use the running mean of interval 20 to make the plot looks smoother. Note that we always use the same lambda for the eligibility trace of w and theta.
<img src="ac_e_trace_compare_2.png" style="max-width:80%; width: 80%; max-width: none">

Discussions and Conclusions:

- compare the 1000 episodes results of REINFORCE without or with baseline, we could see that REINFORCE with baseline is learning better. Within 1000 episodes, REINFORCE without baseline is learning almost nothing, where with baseline, it improved the rewards from -165 to around -135 with 1000.


- compare REINFORCE with actor critic, we could see actor critic performed significantly better than REINFORCE which agrees with what Doina is talking about during the class. REINFORCE use returns obtained from monte carlo which have very high variance, while actor-critic use TD errors and update in an on-policy way. Actor critic with one step return increased rewards from -160 to -60 within 60 episodes which is the best algorithm we  learned so far (this includes the mountain car with linear function approximation as what we did last week).


- Looking at actor critic with eligibility trace. we could see that intermediate lambda values (for both w and theta) performed the best. When lambda =1, the algorithm is almost equivalent as REINFORCE (I think it is equivalent as without baseline, but I am not sure) and it learned almost nothing within 200 episodes. On the other hand, when lambda =0, it is equivalent as one step actor critic. We could see that lambda=0.25,0.5,or 0.75 performed the best.  